Pre-Processing

In [ ]:
import cv2
import numpy as np
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount = True)

# Define the folder path
folder_path = '/content/drive/MyDrive/Phase 2/normal set 1'

# Define the output folder path
output_folder_path = '/content/drive/MyDrive/Phase 2/PP1'

# Create the output folder if it doesn't exist
!mkdir -p "$output_folder_path"

# Function to perform noise removal using Gaussian blurring
def remove_noise(image):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    return blurred

# Function to adjust brightness and contrast using gamma transformations
def adjust_brightness_contrast(image, brightness=0.5, contrast=0.5):
    adjusted = np.power(image / 255.0, 1 / contrast) * (255.0 * brightness)
    return np.clip(adjusted, 0, 255).astype(np.uint8)

# Function to sharpen the edges using unsharp masking
def sharpen_edges(image):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sharpened = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    return sharpened

# Function to resize the image to 512x512 without making it pixelated
def resize_image(image):
    resized = cv2.resize(image, (512, 512), interpolation=cv2.INTER_AREA)
    return resized

# Iterate over images in the folder
for image_name in os.listdir(folder_path):
    # Load the image
    image_path = os.path.join(folder_path, image_name)
    image = cv2.imread(image_path)

    # Remove noise using Gaussian blurring
    image = remove_noise(image)

    # Adjust brightness and contrast using gamma transformations
    image = adjust_brightness_contrast(image, brightness=1.2, contrast=1.5)

    # Sharpen the edges using unsharp masking
    image = sharpen_edges(image)

    # Resize the image to 512x512
    image = resize_image(image)

    # Save the processed image
    output_path = os.path.join(output_folder_path, image_name)
    cv2.imwrite(output_path, image)


Masking pre-processed images

In [ ]:
import os
import cv2
import numpy as np

# Define the curve points for the damage
curve_points = np.array([[(100, 100), (200, 300), (400, 200), (500, 100)]], dtype=np.int32)

# Set the input and output directories
input_dir = '/content/drive/MyDrive/Phase 2/PP1-Combined'
output_dir = '/content/drive/MyDrive/Phase 2/PP1-Combined Masked'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through all the files in the input directory
for filename in os.listdir(input_dir):
    # Load the input image
    img_path = os.path.join(input_dir, filename)
    img = cv2.imread(img_path)

    # Create a copy of the input image
    damaged_img = img.copy()

    # Draw many small curves on the damaged image
    for i in range(10):
        curve_points = np.random.randint(0, img.shape[0], size=(4, 2))
        cv2.polylines(damaged_img, [curve_points], False, (255, 255, 255), thickness=7)

    # Save the damaged image in the output directory
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, damaged_img)

Sub-image Generation

In [ ]:
import os
import imageio
from PIL import Image

# Function to upscale and upsize the image
def upscale_image(image_path, output_path):
    image = Image.fromarray(image_path)
    upscaled_image = image.resize((512, 512), Image.BICUBIC)
    upscaled_image.save(output_path)

# Function to divide the image into four equal parts
def divide_image(image_path, output_folder):
    image = imageio.imread(image_path)
    height, width, _ = image.shape
    half_width = width // 2
    half_height = height // 2

    # Dividing the image into four parts
    top_left = image[:half_height, :half_width]
    top_right = image[:half_height, half_width:]
    bottom_left = image[half_height:, :half_width]
    bottom_right = image[half_height:, half_width:]

    # Creating output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get the original filename without extension
    filename = os.path.splitext(os.path.basename(image_path))[0]

    # Upscaling and saving each divided image
    upscale_image(top_left, os.path.join(output_folder, f'{filename}_top_left.png'))
    upscale_image(top_right, os.path.join(output_folder, f'{filename}_top_right.png'))
    upscale_image(bottom_left, os.path.join(output_folder, f'{filename}_bottom_left.png'))
    upscale_image(bottom_right, os.path.join(output_folder, f'{filename}_bottom_right.png'))

# Path to the folder containing input images
input_folder = '/content/drive/MyDrive/Phase 2/PP1'

# Path to the output folder for the upscaled images
output_folder = '/content/drive/MyDrive/Phase 2/PP1-SIG'

# Iterate over each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        image_path = os.path.join(input_folder, filename)
        divide_image(image_path, output_folder)


<ipython-input-11-e384ab439bbb>:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


Model Training

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.image import psnr as psnr_func
import os
import numpy as np

def partial_conv_block(x, channels, kernel_size=3, strides=1, padding='same', use_bias=False, alpha=0.2, use_bn=True):
    # Define a partial convolutional layer with mask
    x_in = x
    x = Conv2D(channels, kernel_size, strides=strides, padding=padding, use_bias=use_bias)(x)
    if use_bn:
        x = BatchNormalization()(x)
    mask = Conv2D(channels, kernel_size, strides=strides, padding=padding, use_bias=use_bias,
                  activation='sigmoid')(x_in)
    # Compute the element-wise multiplication between output feature maps and mask
    x_out = multiply([x, mask])
    # Apply LeakyReLU activation function
    x_out = LeakyReLU(alpha=alpha)(x_out)
    return x_out, mask

def psnr(y_true, y_pred):
    # Define a custom PSNR metric function
    return psnr_func(y_true, y_pred, max_val=1.0)

def unet_part_conv(input_shape, alpha=0.2):
    # Defines the U-Net architecture with partial convolution blocks
    inputs = Input(input_shape)

    # Encoder
    enc1, mask1 = partial_conv_block(inputs, 64, use_bn=False)
    pool1 = MaxPooling2D(pool_size=(2, 2))(mask1)
    enc2, mask2 = partial_conv_block(pool1, 128, use_bn=False)
    pool2 = MaxPooling2D(pool_size=(2, 2))(mask2)
    enc3, mask3 = partial_conv_block(pool2, 256, use_bn=False)
    pool3 = MaxPooling2D(pool_size=(2, 2))(mask3)
    enc4, mask4 = partial_conv_block(pool3, 512, use_bn=False)
    pool4 = MaxPooling2D(pool_size=(2, 2))(mask4)

    # Bottleneck
    center, mask5 = partial_conv_block(pool4, 1024, use_bn=True)

    # Decoder with skip connections
    up1 = Conv2DTranspose(512, 2, strides=2, padding='same')(center)
    concat1 = concatenate([up1, enc4])
    dec1, mask6 = partial_conv_block(concat1, 512, use_bn=True)
    up2 = Conv2DTranspose(256, 2, strides=2, padding='same')(dec1)
    concat2 = concatenate([up2, enc3])
    dec2, mask7 = partial_conv_block(concat2, 256, use_bn=True)
    up3 = Conv2DTranspose(128, 2, strides=2, padding='same')(dec2)
    concat3 = concatenate([up3, enc2])
    dec3, mask8 = partial_conv_block(concat3, 128, use_bn=True)
    up4 = Conv2DTranspose(64, 2, strides=2, padding='same')(dec3)
    concat4 = concatenate([up4, enc1])
    dec4, mask9 = partial_conv_block(concat4, 64, use_bn=True)

    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(dec4)

    # Define model and compile with loss and optimizer
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.00002),
                  loss=binary_crossentropy,
                  metrics=['acc', psnr])
    return model

# Define the path to the directory containing the images
img_dir = '/content/drive/MyDrive/Combined1/'

# Get a list of all image files in the directory
img_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith('.jpg')]

# Split the data into training and validation sets
val_split = 0.2
num_val_samples = int(val_split * len(img_files))
np.random.shuffle(img_files)
train_files = img_files[:-num_val_samples]
val_files = img_files[-num_val_samples:]

# Load training images and masks
train_images = []
train_masks = []
for img_file in train_files:
    # Load input image and mask
    img = tf.io.decode_image(tf.io.read_file(img_file))
    mask_file = os.path.join('/content/drive/MyDrive/CombinedMasked1/', os.path.basename(img_file))
    mask = tf.io.decode_image(tf.io.read_file(mask_file))

    # Reshape input image and mask as per the model requirements
    img = tf.image.resize(img, (512, 512))
    mask = tf.image.resize(mask, (512, 512))
    mask = tf.image.rgb_to_grayscale(mask)
    mask = tf.cast(mask, tf.float32) / 255.

    # Add image and mask to list
    train_images.append(img.numpy())
    train_masks.append(mask.numpy())

# Load validation images and masks
val_images = []
val_masks = []
for img_file in val_files:
    # Load input image and mask
    img = tf.io.decode_image(tf.io.read_file(img_file))
    mask_file = os.path.join('/content/drive/MyDrive/CombinedMasked1/', os.path.basename(img_file))
    mask = tf.io.decode_image(tf.io.read_file(mask_file))

    # Reshape input image and mask as per the model requirements
    img = tf.image.resize(img, (512, 512))
    mask = tf.image.resize(mask, (512, 512))
    mask = tf.image.rgb_to_grayscale(mask)
    mask = tf.cast(mask, tf.float32) / 255.

    # Add image and mask to list
    val_images.append(img.numpy())
    val_masks.append(mask.numpy())

# Convert lists to numpy arrays
train_images = np.array(train_images)
train_masks = np.array(train_masks)
val_images = np.array(val_images)
val_masks = np.array(val_masks)

# Instantiate the model
model = unet_part_conv((512, 512, 3))

# Define the directory to save models in Google Drive
save_dir = '/content/drive/MyDrive/model6/'

# Create the save directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define the callback to save the model after each epoch
save_callback = tf.keras.callbacks.ModelCheckpoint(
    save_dir + 'model_{epoch:03d}.h5')

# Fit the model to the training data and validate on the validation data
model.fit(train_images, train_masks, epochs=20,
          callbacks=[save_callback], batch_size=7,
          validation_data=(val_images, val_masks))

Splitting of oversized damaged images

In [ ]:
from google.colab import drive
import cv2
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the directory containing the images to split
image_dir = '/content/drive/MyDrive/Datasets/Damaged/Damaged Mask/'
# Define the path to the directory to save the split images
output_dir = '/content/drive/MyDrive/Phase 2/split_images1'

# Define the size of the patches
patch_size = 512

# Loop over each image in the input directory
for filename in os.listdir(image_dir):
    # Load the image
    img = cv2.imread(os.path.join(image_dir, filename))

    # Get the dimensions of the image
    height, width, channels = img.shape

    # Calculate the number of rows and columns of patches
    rows = height // patch_size
    cols = width // patch_size

    # Loop over each patch and save it as a separate image
    for i in range(rows):
        for j in range(cols):
            # Calculate the coordinates of the patch
            x = j * patch_size
            y = i * patch_size

            # Extract the patch from the image
            patch = img[y:y+patch_size, x:x+patch_size]

            # Save the patch to disk
            output_filename = os.path.splitext(filename)[0] + f'_{i}_{j}.jpg'
            cv2.imwrite(os.path.join(output_dir, output_filename), patch)

Testing

In [ ]:
from google.colab import drive
import h5py
import cv2
import os
import tensorflow as tf
from tensorflow.image import psnr as psnr_func

# Define the custom PSNR metric function
def psnr(y_true, y_pred):
    max_pixel_value = 255.0
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    psnr = 20 * tf.math.log(max_pixel_value / tf.math.sqrt(mse)) / tf.math.log(10.0)
    return psnr

# Register the custom metric with Keras
tf.keras.metrics.psnr = psnr

psnr_values = []

# Mount Google Drive
drive.mount('/content/drive',force_remount=True)

# Set the input and output file paths
input_dir = '/content/drive/MyDrive/Phase 2/split_images1'
output_dir = '/content/drive/MyDrive/Phase 2/Inpainted'
model_path = '/content/drive/MyDrive/model8/model_020.h5'

# Load the Keras model and custom PSNR function
with tf.keras.utils.custom_object_scope({'psnr': psnr}):
    model = tf.keras.models.load_model(model_path)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['psnr'])

# Loop over all the image files in the input directory
for filename in os.listdir(input_dir):
    # Load the image
    img = cv2.imread(os.path.join(input_dir, filename))

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Threshold the image to obtain a mask of the white areas
    mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)[1]

    # Inpaint the white areas using the cv2.INPAINT_TELEA method
    inpaint = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)

    # Predict the output using the loaded model
    output = model.predict(tf.expand_dims(inpaint, axis=0))[0]

    cv2.imwrite(os.path.join(output_dir, filename[:-4] + '.jpg'), inpaint)